In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Cargar el dataset Malaria
dataset, ds_info = tfds.load('malaria', as_supervised=True, with_info=True)

# Cargar el conjunto de entrenamiento
train_dataset = dataset['train']

# Dividir el conjunto de entrenamiento en entrenamiento y validación (80% y 20%)
train_size = int(0.8 * ds_info.splits['train'].num_examples)  # 80% para entrenamiento
val_size = ds_info.splits['train'].num_examples - train_size  # 20% para validación

# Dividir el dataset
train_dataset = train_dataset.take(train_size)
val_dataset = dataset['train'].skip(train_size)  # Asegúrate de que el conjunto de validación se defina correctamente

# Cargar el conjunto de prueba (20% del dataset original)
test_dataset = dataset['test']  # Cargamos el conjunto de prueba

# Función de preprocesamiento
def preprocess_image(image, label):
    image = tf.image.resize(image, [128, 128])  # Redimensionar imágenes
    image = image / 255.0  # Normalizar imágenes
    return image, label

# Preprocesar los datasets
train_dataset = train_dataset.map(preprocess_image).batch(32)
val_dataset = val_dataset.map(preprocess_image).batch(32)

# Asegúrate de que test_dataset esté definido antes de procesarlo
test_dataset = test_dataset.map(preprocess_image).batch(32)  # Preprocesar el conjunto de prueba

# Visualización de algunas imágenes del dataset
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):  # Mostrar 9 imágenes
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])  # Acceder a cada imagen en el lote
        plt.title(f"Label: {'Parasitized' if labels[i] == 1 else 'Uninfected'}")
        plt.axis('off')
plt.show()

# Crear la red neuronal convolucional
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # Dos clases: Parasitized y Uninfected
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_dataset, epochs=5, validation_data=val_dataset)

# Guardar el modelo
model.save('malaria_cnn_model.h5')

# Hacer predicciones y evaluación del modelo
predictions = model.predict(test_dataset)

# Obtener las etiquetas verdaderas para el conjunto de prueba
y_true = []
y_pred = []

for images, labels in test_dataset:
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(model.predict(images), axis=1))

# Mostrar el reporte de clasificación
print(classification_report(y_true, y_pred))

# Crear y mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Uninfected', 'Parasitized'], yticklabels=['Uninfected', 'Parasitized'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]